In [1]:
from search_travessia_rio import *
import json
import os
import webbrowser
import http.server
import socketserver
import threading

In [2]:
#Definição formal do problema travessia do rio
grafo = {
    (0,0,0,0,0,0,0,0,0): [(1,0,0,1,0,0,0,0,1), (0,0,0,0,0,0,1,1,1)],
    (1,0,0,1,0,0,0,0,1): [(0,0,0,0,0,0,0,0,0)],
    (0,0,0,0,0,0,1,1,1): [(0,0,0,0,0,0,0,0,0), (0,0,0,0,0,0,0,1,0)],
    (0,0,0,0,0,0,0,1,0): [(0,0,0,0,0,0,1,1,1), (0,1,0,0,0,0,1,1,1), (0,0,1,0,0,0,1,1,1), (0,0,0,0,1,0,1,1,1), (0,0,0,0,0,1,1,1,1)],
    (0,1,0,0,0,0,1,1,1): [(0,0,0,0,0,0,0,1,0), (0,1,0,0,0,0,0,0,0)],
    (0,0,1,0,0,0,1,1,1): [(0,0,0,0,0,0,0,1,0), (0,0,1,0,0,0,0,0,0)],
    (0,0,0,0,1,0,1,1,1): [(0,0,0,0,0,0,0,1,0), (0,0,0,0,1,0,0,0,0)],
    (0,0,0,0,1,0,0,0,0): [(1,0,0,1,1,0,0,0,1), (0,0,0,0,1,0,1,1,1), (0,0,0,1,1,0,0,0,1), (0,0,0,1,1,1,0,0,1)],
    (0,0,0,1,1,0,0,0,1): [(0,0,0,0,1,0,0,0,0), (0,0,0,0,0,0,0,0,0)],
    (0,0,0,0,0,1,1,1,1): [(0,0,0,0,0,0,0,1,0), (0,0,0,0,0,1,0,0,0)],
    (0,1,0,0,0,0,0,0,0): [(1,1,1,0,0,0,0,0,1), (0,1,0,0,0,0,1,1,1), (1,1,0,1,0,0,0,0,1)],
    (0,0,1,0,0,0,0,0,0): [(1,1,1,0,0,0,0,0,1), (0,0,1,0,0,0,1,1,1), (1,0,0,1,1,0,0,0,1)],
    (0,0,0,0,0,1,0,0,0): [(0,0,0,1,1,1,0,0,1), (0,0,0,0,0,1,1,1,1), (1,0,0,1,0,1,0,0,1)],
    (1,1,1,0,0,0,0,0,1): [(0,0,1,0,0,0,0,0,0), (0,1,0,0,0,0,0,0,0), (0,1,1,0,0,0,0,0,0)],
    (1,1,0,1,0,0,0,0,1): [(0,1,0,0,0,0,0,0,0)],
    (1,0,1,1,0,0,0,0,1): [(0,0,1,0,0,0,0,0,0)],
    (0,0,0,1,1,1,0,0,1): [(0,0,0,0,1,1,0,0,0), (0,0,0,0,0,1,0,0,0), (0,0,0,0,1,0,0,0,0)],
    (1,0,0,1,1,0,0,0,1): [(0,0,0,0,1,0,0,0,0)],
    (1,0,0,1,0,1,0,0,1): [(0,0,0,0,0,1,0,0,0)],
    (0,1,1,0,0,0,0,0,0): [(1,1,1,0,0,0,0,0,1), (1,1,1,1,0,0,0,0,1), (0,1,1,0,0,0,1,1,1)],
    (0,0,0,0,1,1,0,0,0): [(0,0,0,1,1,1,0,0,1), (1,0,0,1,1,1,0,0,1), (0,0,0,0,1,1,1,1,1)],
    (1,1,1,1,0,0,0,0,1): [(1,1,1,0,0,0,0,0,0), (0,1,1,0,0,0,0,0,0)],
    (0,1,1,0,0,0,1,1,1): [(0,1,1,0,0,0,0,0,0)],
    (1,0,0,1,1,1,0,0,1): [(0,0,0,1,1,1,0,0,0), (0,0,0,0,1,1,0,0,0)],
    (0,0,0,0,1,1,1,1,1): [(0,0,0,0,1,1,0,0,0)],
    (1,1,1,0,0,0,0,0,0): [(1,1,1,1,1,0,0,0,1), (1,1,1,1,0,1,0,0,1), (1,1,1,1,0,0,0,0,1), (1,1,1,0,0,0,1,1,1)],
    (0,0,0,1,1,1,0,0,0): [(1,1,0,1,1,1,0,0,1), (1,0,1,1,1,1,0,0,1), (1,0,0,1,1,1,0,0,1), (0,0,0,1,1,1,1,1,1)],
    (1,1,1,1,1,0,0,0,1): [(0,1,1,0,1,0,0,0,0), (1,1,1,0,0,0,0,0,0)],
    (1,1,1,1,0,1,0,0,1): [(0,1,1,0,0,1,0,0,0), (1,1,1,0,0,0,0,0,0)],
    (1,1,1,0,0,0,1,1,1): [(0,1,1,0,0,0,1,1,0), (0,0,1,0,0,0,1,1,0), (0,1,0,0,0,0,1,1,0), (1,1,1,0,0,0,0,0,0)],
    (1,1,0,1,1,1,0,0,1): [(0,0,0,1,1,1,0,0,0), (0,1,0,0,1,1,0,0,0)],
    (1,0,1,1,1,1,0,0,1): [(0,0,0,1,1,1,0,0,0), (0,0,1,0,1,1,0,0,0)],
    (0,0,0,1,1,1,1,1,1): [(0,0,0,0,1,1,1,1,0), (0,0,0,0,0,1,1,1,0), (0,0,0,0,1,0,1,1,0), (0,0,0,1,1,1,0,0,0)],
    (0,1,1,0,1,0,0,0,0): [(1,1,1,1,1,0,0,0,1), (0,1,1,0,1,0,1,1,1)],
    (0,1,1,0,0,1,0,0,0): [(1,1,1,1,0,1,0,0,1), (0,1,1,0,0,1,1,1,1)],
    (0,1,1,0,0,0,1,1,0): [(1,1,1,0,0,0,1,1,1), (1,1,1,1,0,0,1,1,1)],
    (0,0,1,0,0,0,1,1,0): [(1,1,1,0,0,0,1,1,1), (1,0,1,1,0,0,1,1,1)],
    (0,1,0,0,0,0,1,1,0): [(1,1,1,0,0,0,1,1,1), (1,1,0,1,0,0,1,1,1)],
    (0,1,0,0,1,1,0,0,0): [(1,1,0,1,1,1,0,0,1), (0,1,0,0,1,1,1,1,1)],
    (0,0,1,0,1,1,0,0,0): [(1,0,1,1,1,1,0,0,1), (0,0,1,0,1,1,1,1,1)],
    (0,0,0,0,1,1,1,1,0): [(1,0,0,1,1,1,1,1,1), (0,0,0,1,1,1,1,1,1)],
    (0,0,0,0,0,1,1,1,0): [(1,0,0,1,0,1,1,1,1), (0,0,0,1,1,1,1,1,1)],
    (0,0,0,0,1,0,1,1,0): [(1,0,0,1,1,0,1,1,1), (0,0,0,1,1,1,1,1,1)],
    (0,1,1,0,1,0,1,1,1): [(0,1,1,0,1,0,0,0,0)],
    (0,1,1,0,0,1,1,1,1): [(0,1,1,0,0,1,0,0,0)],
    (1,1,1,1,0,0,1,1,1): [(0,1,1,0,0,0,1,1,0), (1,1,1,0,0,0,1,1,0), (1,1,1,1,0,0,0,0,0)],
    (1,0,1,1,0,0,1,1,1): [(0,0,1,0,0,0,1,1,0), (1,0,1,1,0,0,0,0,0)],
    (1,1,0,1,0,0,1,1,1): [(0,1,0,0,0,0,1,1,0), (1,1,0,1,0,0,0,0,0)],
    (0,1,0,0,1,1,1,1,1): [(0,1,0,0,1,1,0,0,0)],
    (0,0,1,0,1,1,1,1,1): [(0,0,1,0,1,1,0,0,0)],
    (1,0,0,1,1,1,1,1,1): [(0,0,0,1,1,1,1,1,0), (1,0,0,1,1,1,0,0,0), (0,0,0,0,1,1,1,1,1)],
    (1,0,0,1,0,1,1,1,1): [(0,0,0,0,0,1,1,1,0), (1,0,0,1,0,1,0,0,0)],
    (1,0,0,1,1,0,1,1,1): [(0,0,0,0,1,0,1,1,0), (1,0,0,1,1,0,0,0,0)],
    (1,1,1,0,0,0,1,1,0): [(1,1,1,1,0,0,1,1,1), (1,1,1,1,1,0,1,1,1), (1,1,1,1,0,1,1,1,1),],
    (1,1,1,1,0,0,0,0,0): [(1,1,1,1,0,0,1,1,1)],
    (1,0,1,1,0,0,0,0,0): [(1,0,1,1,0,0,1,1,1)],
    (1,1,0,1,0,0,0,0,0): [(1,1,0,1,0,0,1,1,1)],
    (0,0,0,1,1,1,1,1,0): [(1,0,0,1,1,1,1,1,1), (1,1,0,1,1,1,1,1,1), (1,0,1,1,1,1,1,1,1)],
    (1,0,0,1,1,1,0,0,0): [(1,0,0,1,1,1,1,1,1)],
    (1,0,0,1,0,1,0,0,0): [(1,0,0,1,0,1,1,1,1)],
    (1,0,0,1,1,0,0,0,0): [(1,0,0,1,1,0,1,1,1)],
    (1,1,1,1,1,0,1,1,1): [(1,1,1,0,0,0,1,1,0), (1,1,1,1,1,0,0,0,0)],
    (1,1,1,1,0,1,1,1,1): [(1,1,1,0,0,0,1,1,0), (1,1,1,1,0,1,0,0,0)],
    (1,1,0,1,1,1,1,1,1): [(0,0,0,1,1,1,1,1,0), (1,1,0,1,1,1,0,0,0)],
    (1,0,1,1,1,1,1,1,1): [(0,0,0,1,1,1,1,1,0), (1,0,1,1,1,1,0,0,0)],
    (1,1,1,1,1,0,0,0,0): [(1,1,1,1,1,1,1,0,1), (1,1,1,1,1,0,1,1,1)],
    (1,1,1,1,0,1,0,0,0): [(1,1,1,1,1,1,1,0,1), (1,1,1,1,0,1,1,1,1)],
    (1,1,0,1,1,1,0,0,0): [(1,1,1,1,1,1,1,0,1), (1,1,0,1,1,1,1,1,1)],
    (1,0,1,1,1,1,0,0,0): [(1,1,1,1,1,1,1,0,1), (1,0,1,1,1,1,1,1,1)],
    (1,1,1,1,1,1,1,0,1): [(1,1,1,1,1,1,0,0,0), (1,0,1,1,1,1,0,0,0), (1,1,0,1,1,1,0,0,0), (1,1,1,1,0,1,0,0,0), (1,1,1,1,1,0,0,0,0)],
    (1,1,1,1,1,1,0,0,0): [(1,1,1,1,1,1,1,0,1), (1,1,1,1,1,1,1,1,1)],
    (1,1,1,1,1,1,1,1,1): [(0,1,1,0,1,1,1,1,0), (1,1,1,1,1,1,0,0,0)]
}

estadoInicial = (0,0,0,0,0,0,0,0,0)

estadoFinal = (1,1,1,1,1,1,1,1,1)

travessia_rio = Problem(estadoInicial, estadoFinal, grafo)

In [3]:
#Busca em largura
print("Camino percorrido para busca em largura:")
resultLargura = busca_em_largura(travessia_rio)
custoDeCaminho = 0
resposta = []
for node in resultLargura:
    resposta.append(node.state)
    custoDeCaminho += 1
    print(node.state)
print(f"Custo do caminho: {custoDeCaminho}")

caminho_pasta = "animacao"  
    
if not os.path.exists(caminho_pasta):
    os.makedirs(caminho_pasta)
    
nome_arquivo = "dados.json"
    
caminho_arquivo = os.path.join(caminho_pasta, nome_arquivo)

with open(caminho_arquivo, "w") as arquivo:
    json.dump(resposta, arquivo)
    
def abrir_navegador(url):
    
    webbrowser.open(url)

def iniciar_servidor(diretorio):
        # Obter o diretório absoluto do servidor
    diretorio_absoluto = os.path.abspath(diretorio)

        # Configurar o servidor web local para servir arquivos a partir do diretório específico
    porta = 8000
    os.chdir(diretorio_absoluto)  # Mudar para o diretório de onde os arquivos serão servidos
    handler = http.server.SimpleHTTPRequestHandler

        # Desativar o bloqueio de endereço já em uso para reinicializações rápidas
    socketserver.TCPServer.allow_reuse_address = True

        # Iniciar o servidor web local
    httpd = socketserver.TCPServer(("", porta), handler)
    print(f"Servidor web iniciado em http://localhost:{porta}/animacao/")
        
        # Abrir o navegador no URL especificado após iniciar o servidor
    abrir_navegador(f"http://localhost:{porta}")

        # Servir os arquivos continuamente até que o servidor seja interrompido
    httpd.serve_forever()
diretorio_servidor = "animacao"

    # Iniciar o servidor web local em uma thread separada
servidor_thread = threading.Thread(target=iniciar_servidor, args=(diretorio_servidor,))
servidor_thread.start()

    # Aguardar a execução principal terminar para encerrar o servidor web
servidor_thread.join()

Camino percorrido para busca em largura:
Quantidade de movimentos: 130
(0, 0, 0, 0, 0, 0, 0, 0, 0)
(0, 0, 0, 0, 0, 0, 1, 1, 1)
(0, 0, 0, 0, 0, 0, 0, 1, 0)
(0, 1, 0, 0, 0, 0, 1, 1, 1)
(0, 1, 0, 0, 0, 0, 0, 0, 0)
(1, 1, 1, 0, 0, 0, 0, 0, 1)
(0, 1, 1, 0, 0, 0, 0, 0, 0)
(1, 1, 1, 1, 0, 0, 0, 0, 1)
(1, 1, 1, 0, 0, 0, 0, 0, 0)
(1, 1, 1, 0, 0, 0, 1, 1, 1)
(0, 1, 1, 0, 0, 0, 1, 1, 0)
(1, 1, 1, 1, 0, 0, 1, 1, 1)
(1, 1, 1, 0, 0, 0, 1, 1, 0)
(1, 1, 1, 1, 1, 0, 1, 1, 1)
(1, 1, 1, 1, 1, 0, 0, 0, 0)
(1, 1, 1, 1, 1, 1, 1, 0, 1)
(1, 1, 1, 1, 1, 1, 0, 0, 0)
(1, 1, 1, 1, 1, 1, 1, 1, 1)
Custo do caminho: 18
Servidor web iniciado em http://localhost:8000/animacao/


127.0.0.1 - - [29/Apr/2024 11:28:49] "GET / HTTP/1.1" 304 -
127.0.0.1 - - [29/Apr/2024 11:28:49] "GET /cara.png HTTP/1.1" 304 -
127.0.0.1 - - [29/Apr/2024 11:28:49] "GET /dados.json HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2024 11:28:49] "GET /barco.png HTTP/1.1" 304 -
127.0.0.1 - - [29/Apr/2024 11:28:49] "GET /terrorista.png HTTP/1.1" 304 -
127.0.0.1 - - [29/Apr/2024 11:28:49] "GET /garoto.png HTTP/1.1" 304 -
127.0.0.1 - - [29/Apr/2024 11:28:49] "GET /garoto%20(1).png HTTP/1.1" 304 -
127.0.0.1 - - [29/Apr/2024 11:28:49] "GET /mae.png HTTP/1.1" 304 -
127.0.0.1 - - [29/Apr/2024 11:28:50] "GET /menina.png HTTP/1.1" 304 -
127.0.0.1 - - [29/Apr/2024 11:28:50] "GET /menina%20(1).png HTTP/1.1" 304 -
127.0.0.1 - - [29/Apr/2024 11:28:50] "GET /policial.png HTTP/1.1" 304 -


: 

In [3]:
#Busca em profundidade
print("Camino percorrido para busca em profundidade:")
resultProfundidade = busca_em_profundidade(travessia_rio)
custoDeCaminho = 0
resposta = []
for node in resultProfundidade:
    resposta.append(node.state)
    custoDeCaminho += 1
    print(node.state)
print(f"Custo do caminho: {custoDeCaminho}")

caminho_pasta = "animacao"  
    
if not os.path.exists(caminho_pasta):
    os.makedirs(caminho_pasta)
    
nome_arquivo = "dados.json"
    
caminho_arquivo = os.path.join(caminho_pasta, nome_arquivo)

with open(caminho_arquivo, "w") as arquivo:
    json.dump(resposta, arquivo)
    
def abrir_navegador(url):
    
    webbrowser.open(url)

def iniciar_servidor(diretorio):
        # Obter o diretório absoluto do servidor
    diretorio_absoluto = os.path.abspath(diretorio)

        # Configurar o servidor web local para servir arquivos a partir do diretório específico
    porta = 8000
    os.chdir(diretorio_absoluto)  # Mudar para o diretório de onde os arquivos serão servidos
    handler = http.server.SimpleHTTPRequestHandler

        # Desativar o bloqueio de endereço já em uso para reinicializações rápidas
    socketserver.TCPServer.allow_reuse_address = True

        # Iniciar o servidor web local
    httpd = socketserver.TCPServer(("", porta), handler)
    print(f"Servidor web iniciado em http://localhost:{porta}/animacao/")
        
        # Abrir o navegador no URL especificado após iniciar o servidor
    abrir_navegador(f"http://localhost:{porta}")

        # Servir os arquivos continuamente até que o servidor seja interrompido
    httpd.serve_forever()
diretorio_servidor = "animacao"

    # Iniciar o servidor web local em uma thread separada
servidor_thread = threading.Thread(target=iniciar_servidor, args=(diretorio_servidor,))
servidor_thread.start()

    # Aguardar a execução principal terminar para encerrar o servidor web
servidor_thread.join()

Camino percorrido para busca em profundidade:
Quantidade de movimentos: 65
(0, 0, 0, 0, 0, 0, 0, 0, 0)
(0, 0, 0, 0, 0, 0, 1, 1, 1)
(0, 0, 0, 0, 0, 0, 0, 1, 0)
(0, 0, 0, 0, 0, 1, 1, 1, 1)
(0, 0, 0, 0, 0, 1, 0, 0, 0)
(0, 0, 0, 1, 1, 1, 0, 0, 1)
(0, 0, 0, 0, 1, 1, 0, 0, 0)
(1, 0, 0, 1, 1, 1, 0, 0, 1)
(0, 0, 0, 1, 1, 1, 0, 0, 0)
(0, 0, 0, 1, 1, 1, 1, 1, 1)
(0, 0, 0, 0, 1, 1, 1, 1, 0)
(1, 0, 0, 1, 1, 1, 1, 1, 1)
(0, 0, 0, 1, 1, 1, 1, 1, 0)
(1, 0, 1, 1, 1, 1, 1, 1, 1)
(1, 0, 1, 1, 1, 1, 0, 0, 0)
(1, 1, 1, 1, 1, 1, 1, 0, 1)
(1, 1, 1, 1, 1, 1, 0, 0, 0)
(1, 1, 1, 1, 1, 1, 1, 1, 1)
Custo do caminho: 18
Servidor web iniciado em http://localhost:8000/animacao/


127.0.0.1 - - [29/Apr/2024 11:24:09] "GET / HTTP/1.1" 304 -
127.0.0.1 - - [29/Apr/2024 11:24:09] "GET /cara.png HTTP/1.1" 304 -
127.0.0.1 - - [29/Apr/2024 11:24:09] "GET /dados.json HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2024 11:24:09] "GET /barco.png HTTP/1.1" 304 -
127.0.0.1 - - [29/Apr/2024 11:24:09] "GET /terrorista.png HTTP/1.1" 304 -
127.0.0.1 - - [29/Apr/2024 11:24:09] "GET /garoto.png HTTP/1.1" 304 -
127.0.0.1 - - [29/Apr/2024 11:24:09] "GET /garoto%20(1).png HTTP/1.1" 304 -
127.0.0.1 - - [29/Apr/2024 11:24:09] "GET /mae.png HTTP/1.1" 304 -
127.0.0.1 - - [29/Apr/2024 11:24:09] "GET /menina.png HTTP/1.1" 304 -
127.0.0.1 - - [29/Apr/2024 11:24:09] "GET /menina%20(1).png HTTP/1.1" 304 -
127.0.0.1 - - [29/Apr/2024 11:24:09] "GET /policial.png HTTP/1.1" 304 -


: 

In [8]:
#Busca em profundidade limitada
print("Caminho percorrido para busca em profundidade limitada")
resultProfundidadeLimitada = busca_em_profundidade_limitada(travessia_rio, 17)
custoDeCaminho = 0
resposta = []
for node in resultProfundidadeLimitada:
    try:
        resposta.append(node.state)
        print(node.state)
        custoDeCaminho += 1
    except AttributeError: 
        print(node) 
print(f"Custo do caminho: {custoDeCaminho}")

caminho_pasta = "animacao"  
    
if not os.path.exists(caminho_pasta):
    os.makedirs(caminho_pasta)
    
nome_arquivo = "dados.json"
    
caminho_arquivo = os.path.join(caminho_pasta, nome_arquivo)

with open(caminho_arquivo, "w") as arquivo:
    json.dump(resposta, arquivo)
    
def abrir_navegador(url):
    
    webbrowser.open(url)

def iniciar_servidor(diretorio):
        # Obter o diretório absoluto do servidor
    diretorio_absoluto = os.path.abspath(diretorio)

        # Configurar o servidor web local para servir arquivos a partir do diretório específico
    porta = 8000
    os.chdir(diretorio_absoluto)  # Mudar para o diretório de onde os arquivos serão servidos
    handler = http.server.SimpleHTTPRequestHandler

        # Desativar o bloqueio de endereço já em uso para reinicializações rápidas
    socketserver.TCPServer.allow_reuse_address = True

        # Iniciar o servidor web local
    httpd = socketserver.TCPServer(("", porta), handler)
    print(f"Servidor web iniciado em http://localhost:{porta}/animacao/")
        
        # Abrir o navegador no URL especificado após iniciar o servidor
    abrir_navegador(f"http://localhost:{porta}")

        # Servir os arquivos continuamente até que o servidor seja interrompido
    httpd.serve_forever()
diretorio_servidor = "animacao"

    # Iniciar o servidor web local em uma thread separada
servidor_thread = threading.Thread(target=iniciar_servidor, args=(diretorio_servidor,))
servidor_thread.start()

    # Aguardar a execução principal terminar para encerrar o servidor web
servidor_thread.join()


Caminho percorrido para busca em profundidade limitada
(0, 0, 0, 0, 0, 0, 0, 0, 0)
(0, 0, 0, 0, 0, 0, 1, 1, 1)
(0, 0, 0, 0, 0, 0, 0, 1, 0)
(0, 1, 0, 0, 0, 0, 1, 1, 1)
(0, 1, 0, 0, 0, 0, 0, 0, 0)
(1, 1, 1, 0, 0, 0, 0, 0, 1)
(0, 1, 1, 0, 0, 0, 0, 0, 0)
(1, 1, 1, 1, 0, 0, 0, 0, 1)
(1, 1, 1, 0, 0, 0, 0, 0, 0)
(1, 1, 1, 0, 0, 0, 1, 1, 1)
(0, 1, 1, 0, 0, 0, 1, 1, 0)
(1, 1, 1, 1, 0, 0, 1, 1, 1)
(1, 1, 1, 0, 0, 0, 1, 1, 0)
(1, 1, 1, 1, 1, 0, 1, 1, 1)
(1, 1, 1, 1, 1, 0, 0, 0, 0)
(1, 1, 1, 1, 1, 1, 1, 0, 1)
(1, 1, 1, 1, 1, 1, 0, 0, 0)
(1, 1, 1, 1, 1, 1, 1, 1, 1)
Custo do caminho: 18


In [9]:
#Busca em profundidade limitada interativa
print("Caminho percorrido para busca em profundidade limitada interativa")
resultProfundidadeInterativa = busca_em_profundidade_limitada_interativa(travessia_rio)
custoDeCaminho = 0
resposta = []
for node in resultProfundidadeInterativa:
    try:
        resposta.append(node.state)
        print(node.state)
        custoDeCaminho += 1
    except AttributeError: 
        print(node)
print(f"Custo do caminho: {custoDeCaminho}")

caminho_pasta = "animacao"  
    
if not os.path.exists(caminho_pasta):
    os.makedirs(caminho_pasta)
    
nome_arquivo = "dados.json"
    
caminho_arquivo = os.path.join(caminho_pasta, nome_arquivo)

with open(caminho_arquivo, "w") as arquivo:
    json.dump(resposta, arquivo)
    
def abrir_navegador(url):
    
    webbrowser.open(url)

def iniciar_servidor(diretorio):
        # Obter o diretório absoluto do servidor
    diretorio_absoluto = os.path.abspath(diretorio)

        # Configurar o servidor web local para servir arquivos a partir do diretório específico
    porta = 8000
    os.chdir(diretorio_absoluto)  # Mudar para o diretório de onde os arquivos serão servidos
    handler = http.server.SimpleHTTPRequestHandler

        # Desativar o bloqueio de endereço já em uso para reinicializações rápidas
    socketserver.TCPServer.allow_reuse_address = True

        # Iniciar o servidor web local
    httpd = socketserver.TCPServer(("", porta), handler)
    print(f"Servidor web iniciado em http://localhost:{porta}/animacao/")
        
        # Abrir o navegador no URL especificado após iniciar o servidor
    abrir_navegador(f"http://localhost:{porta}")

        # Servir os arquivos continuamente até que o servidor seja interrompido
    httpd.serve_forever()
diretorio_servidor = "animacao"

    # Iniciar o servidor web local em uma thread separada
servidor_thread = threading.Thread(target=iniciar_servidor, args=(diretorio_servidor,))
servidor_thread.start()

    # Aguardar a execução principal terminar para encerrar o servidor web
servidor_thread.join()


Caminho percorrido para busca em profundidade limitada interativa
(0, 0, 0, 0, 0, 0, 0, 0, 0)
(0, 0, 0, 0, 0, 0, 1, 1, 1)
(0, 0, 0, 0, 0, 0, 0, 1, 0)
(0, 1, 0, 0, 0, 0, 1, 1, 1)
(0, 1, 0, 0, 0, 0, 0, 0, 0)
(1, 1, 1, 0, 0, 0, 0, 0, 1)
(0, 1, 1, 0, 0, 0, 0, 0, 0)
(1, 1, 1, 1, 0, 0, 0, 0, 1)
(1, 1, 1, 0, 0, 0, 0, 0, 0)
(1, 1, 1, 0, 0, 0, 1, 1, 1)
(0, 1, 1, 0, 0, 0, 1, 1, 0)
(1, 1, 1, 1, 0, 0, 1, 1, 1)
(1, 1, 1, 0, 0, 0, 1, 1, 0)
(1, 1, 1, 1, 1, 0, 1, 1, 1)
(1, 1, 1, 1, 1, 0, 0, 0, 0)
(1, 1, 1, 1, 1, 1, 1, 0, 1)
(1, 1, 1, 1, 1, 1, 0, 0, 0)
(1, 1, 1, 1, 1, 1, 1, 1, 1)
Custo do caminho: 18
Servidor web iniciado em http://localhost:8000/animacao/


127.0.0.1 - - [27/Apr/2024 22:43:27] "GET /dados.json HTTP/1.1" 200 -
